# 风险管理作业5
-----
2501210025 叶云鹏

## (一) 
对于某一分布，使用GPD分布来近似超过某一阈值的超额部分的分布。论述对该分布的近似原理，以及推导利用该近似原理估计VaR以及ES的公式


### (一) 使用GPD分布近似超过阈值的超额分布的原理及VaR、ES估计公式推导

#### 1. 近似原理的论述

在极值理论（Extreme Value Theory, EVT）中，对于一个随机变量$X$的分布函数$F(x) = P(X \leq x)$，其尾部行为（即极端事件）往往难以直接建模，尤其是金融损失或风险数据通常呈现重尾特性（heavy-tailed）。为了精确捕捉尾部分布，采用峰值超阈值（Peaks Over Threshold, POT）方法：选择一个足够高的阈值$u$（threshold），关注超过$u$的超额（excess）$Y = X - u \mid X > u$。超额分布函数定义为：
$$
F_u(y) = P(X - u \leq y \mid X > u) = \frac{F(u + y) - F(u)}{1 - F(u)}, \quad y \geq 0,
$$
其中$1 - F(u) = \bar{F}(u)$表示尾概率。

根据Pickands-Balkema-de Haan定理（Pickands, 1975; Balkema and de Haan, 1974），当阈值$u$足够大时（即$u \to \infty$），对于大多数底层分布$F$，超额分布$F_u(y)$可以用广义帕累托分布（Generalized Pareto Distribution, GPD）近似：
$$
F_u(y) \approx G_{\xi, \sigma}(y) = 1 - \left(1 + \xi \frac{y}{\sigma}\right)^{-1/\xi}, \quad \xi \neq 0,
$$
其中$\xi$是形状参数（shape parameter，决定了尾部重度：$\xi > 0$为重尾，$\xi = 0$为指数尾，$\xi < 0$为有限尾），$\sigma > 0$是尺度参数（scale parameter）。当$\xi = 0$时，退化为指数分布：$G_{0, \sigma}(y) = 1 - e^{-y/\sigma}$。此近似成立的条件是$y \geq 0$且$1 + \xi y / \sigma > 0$。

该原理的核心在于GPD的“分布无关性”（distribution-free property）：它不依赖于底层分布的具体形式，只需通过最大似然估计（MLE）等方法从超过$u$的观测样本中拟合$\hat{\xi}$和$\hat{\sigma}$。在实践中，阈值$u$的选择需权衡偏差（bias）和方差（variance）：$u$过低导致近似失效，过高则样本不足。经验上，可用均值剩余函数（mean excess function）或参数稳定性图检验$u$的有效性。该方法特别适用于金融风险建模，因为它能有效捕捉尾部极端事件，而非假设正态分布的轻尾。

基于此近似，整个尾分布的生存函数（survival function）为：
$$
\bar{F}(x) = P(X > x) = \bar{F}(u) \cdot \bar{G}_{\xi, \sigma}(x - u) = \bar{F}(u) \left(1 + \xi \frac{x - u}{\sigma}\right)^{-1/\xi}, \quad x \geq u,
$$
其中$\bar{F}(u) \approx N_u / n$，$n$为总样本大小，$N_u$为超过$u$的观测数。

#### 2. 利用近似原理估计VaR的公式推导

价值-at-风险（Value at Risk, VaR）定义为置信水平$\alpha$（通常$\alpha = 0.95$或$0.99$）下的分位数：
$$
\text{VaR}_\alpha = q_\alpha(F) = \inf \{ x : F(x) \geq \alpha \} = \inf \{ x : \bar{F}(x) \leq 1 - \alpha \},
$$
即损失超过$\text{VaR}_\alpha$的概率为$1 - \alpha$。对于高分位数($\alpha > F(u)$），尾部概率$1 - \alpha < \bar{F}(u)$，故$x > u$，利用GPD近似：
$$
\bar{F}(x) = \bar{F}(u) \left(1 + \xi \frac{x - u}{\sigma}\right)^{-1/\xi} = 1 - \alpha.
$$
解得：
$$
\left(1 + \xi \frac{x - u}{\sigma}\right)^{-1/\xi} = \frac{1 - \alpha}{\bar{F}(u)},
$$
取两边$- \xi$次幂：
$$
1 + \xi \frac{x - u}{\sigma} = \left( \frac{1 - \alpha}{\bar{F}(u)} \right)^{-\xi},
$$
进而：
$$
\xi \frac{x - u}{\sigma} = \left( \frac{1 - \alpha}{\bar{F}(u)} \right)^{-\xi} - 1,
$$
$$
x - u = \frac{\sigma}{\xi} \left[ \left( \frac{1 - \alpha}{\bar{F}(u)} \right)^{-\xi} - 1 \right].
$$
因此：
$$
\text{VaR}_\alpha = u + \frac{\sigma}{\xi} \left[ \left( \frac{1 - \alpha}{\bar{F}(u)} \right)^{-\xi} - 1 \right], \quad \xi \neq 0.
$$
（当$\xi = 0$时，极限形式为$u - \sigma \ln \left( \frac{1 - \alpha}{\bar{F}(u)} \right)$。）

在实证估计中，$\bar{F}(u) \approx N_u / n$，$\xi, \sigma$由MLE从超额样本估计，故估计公式为：
$$
\hat{\text{VaR}}_\alpha = u + \frac{\hat{\sigma}}{\hat{\xi}} \left[ \left( \frac{n (1 - \alpha)}{N_u} \right)^{-\hat{\xi}} - 1 \right].
$$
此公式通过GPD捕捉尾部分位数，避免了参数分布的整体假设，提高了高分位数估计的准确性。

#### 3. 利用近似原理估计ES的公式推导

预期短缺（Expected Shortfall, ES，也称条件VaR）定义为超过VaR的条件期望：
$$
\text{ES}_\alpha = E[X \mid X > \text{VaR}_\alpha] = \frac{1}{1 - \alpha} \int_{\alpha}^1 q_t(F) \, dt,
$$
其中$q_t(F)$是$t)$-分位数函数。它比VaR更具连贯性（coherent），能反映尾部严重程度。

利用GPD近似，对于$x > u$，尾期望可分解：$\text{ES}_\alpha = \text{VaR}_\alpha + E[Y \mid Y > \text{VaR}_\alpha - u]$，其中$Y \sim \text{GPD}(\xi, \sigma)$。GPD的条件剩余期望（conditional mean excess）为：
$$
E[Y \mid Y > z] = z + \frac{\sigma + \xi z}{1 - \xi}, \quad \xi < 1, \, z \geq 0.
$$
（此由GPD生存函数$\bar{G}(y) = (1 + \xi y / \sigma)^{-1/\xi}$的积分推导：$E[Y \mid Y > z] = z + \frac{\int_z^\infty \bar{G}(y) \, dy}{\bar{G}(z)}$，计算得$\int_z^\infty \bar{G}(y) \, dy = \frac{\sigma + \xi z}{1 - \xi} \bar{G}(z)$。）

令$z = \text{VaR}_\alpha - u$，则：
$$
E[Y \mid Y > z] = z + \frac{\sigma + \xi z}{1 - \xi} = \frac{\sigma + \xi (z + u) - \xi u}{1 - \xi} = \frac{\sigma - \xi u + \xi (\text{VaR}_\alpha)}{1 - \xi}.
$$
因此：
$$
\text{ES}_\alpha = \text{VaR}_\alpha + E[Y \mid Y > z] = \text{VaR}_\alpha + \frac{\sigma - \xi u + \xi \text{VaR}_\alpha}{1 - \xi} = \frac{\text{VaR}_\alpha (1 - \xi) + \sigma - \xi u + \xi \text{VaR}_\alpha}{1 - \xi} = \frac{\text{VaR}_\alpha}{1 - \xi} + \frac{\sigma - \xi u}{1 - \xi}.
$$
（要求$\xi < 1$以确保期望存在。）

代入VaR公式并估计参数，即得：
$$
\hat{\text{ES}}_\alpha = \frac{\hat{\text{VaR}}_\alpha}{1 - \hat{\xi}} + \frac{\hat{\sigma} - \hat{\xi} u}{1 - \hat{\xi}}.
$$
推导突显ES对尾部厚度的敏感性（通过$\xi$），在$\xi \to 0$时退化为指数尾的ES形式。



## (二) 
下载一只股票的数据，考虑该股票的日收益率X的VaR和ES，利用 (一) 的方法估计该只股票的日收益率$VaR_{0.95}$

In [1]:
# 第一步：根据原始数据计算日频对数收益率return_df
import pandas as pd
import os
import numpy as np
def get_data(comb_stock_codes):
    
    raw_data_list = []
    for r,d,fs in os.walk('../data/csmar_close'):
        for f in fs:
            if (not ('[DES]' in f)) and ('txt' in f):
                file_path = os.path.join(r,f)
                #print(file_path)
                raw_data_list.append(pd.read_csv(file_path,
                            sep='\t',
                            dtype={'Stkcd': str})[['Trddt','Stkcd','Adjprcwd']])
    working_data  =  pd.concat(raw_data_list,axis=0).set_index(['Trddt','Stkcd'])
    working_data = working_data[~working_data.index.duplicated(keep='first')]

    # return working_data
    workding_df = working_data.unstack('Stkcd')
    df = workding_df.T.dropna().T
    df.columns = df.columns.get_level_values(1)

    return df[comb_stock_codes]

comb_stock_codes = ['000001']

price_df = get_data(comb_stock_codes)
price_df.index = pd.to_datetime(price_df.index)
price_df
return_df = np.log(price_df.shift(-1)/price_df)


In [2]:
from scipy.stats import genpareto

X = return_df.dropna().values  # 转换为 numpy array，便于处理

# 参数设置
alpha = 0.05  # 尾概率，对应 VaR_{0.95}
threshold_prob = 0.10  # 阈值概率（如10%分位数），平衡偏差与方差

# 步骤1: 确定阈值 u（下尾，低分位数）
u = np.quantile(X, threshold_prob)  # u < 0
p = threshold_prob  # 近似 P(X < u) = threshold_prob

# 步骤2: 提取下尾超额 Y = u - X | X < u（Y > 0）
exceedances_mask = X < u
Y = u - X[exceedances_mask]  # 超额，正值

# 步骤3: 拟合 GPD 到超额 Y
# genpareto.fit(Y, floc=0)：floc=0 固定位置参数为0（GPD 从0开始）
shape, loc, scale = genpareto.fit(Y, floc=0)
xi = shape  # 形状参数
sigma = scale  # 尺度参数

print(f"阈值 u: {u:.6f}")
print(f"形状参数 ξ: {xi:.6f}")
print(f"尺度参数 σ: {sigma:.6f}")
print(f"极值样本数: {len(Y)}")

# 步骤4: 计算 VaR_alpha (下尾)
if abs(xi) < 1e-6:  # 若 xi ≈ 0，退化为指数分布
    q = -sigma * np.log(alpha / p)
else:
    delta = alpha / p
    q = (sigma / xi) * (delta**(-xi) - 1)
VaR_alpha = u - q

# 步骤5: 计算 ES_alpha = E[X | X < VaR_alpha] ≈ u - E[Y | Y > q]
if abs(xi - 1) < 1e-6:  # 若 xi ≈ 1，避免分母为0（重尾情况下需调整）
    print("警告: ξ ≈ 1，ES 可能无限大，使用数值积分或调整")
    ES_alpha = np.nan  # 或使用其他方法
else:
    E_Y_given_Y_gt_q = (q + sigma) / (1 - xi)
    ES_alpha = u - E_Y_given_Y_gt_q

print(f"\nPOT估计 VaR_{1-alpha}: {VaR_alpha:.6f}")
print(f"POT估计 ES_{1-alpha}: {ES_alpha:.6f}")

# 与经验估计比较
empirical_VaR = np.quantile(X, alpha)
empirical_ES = np.mean(X[X < empirical_VaR])  # 条件均值
print(f"经验 VaR_{1-alpha}: {empirical_VaR:.6f}")
print(f"经验 ES_{1-alpha}: {empirical_ES:.6f}")

阈值 u: -0.019551
形状参数 ξ: 0.187286
尺度参数 σ: 0.009233
极值样本数: 121

POT估计 VaR_0.95: -0.026385
POT估计 ES_0.95: -0.039320
经验 VaR_0.95: -0.025877
经验 ES_0.95: -0.039295
